In [ ]:
"""
Bu hücre, öğrenci modellerini (student1, student1_base, student2 ve student2_base) yükler ve behavior_data dataset'inden sorulara cevap üretir.
Modeller GPU'lara dağıtılır, cevaplar üretilir ve sonuçlar CSV dosyasına kaydedilir.
"""

import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import torch
from huggingface_hub import login


login(token="--")

# GPU Ayarları
"""
Kullanılabilir GPU sayısını tespit eder ve modelleri GPU'lara dağıtmak için kullanılır.gpu T4x2 kullanırken gerekli.
"""
n_gpus = torch.cuda.device_count()
print(f"GPU sayısı: {n_gpus}")


"""
Anahtar: model adı, Değer: model yolu.
"""
#yapılan modellerin huggingface 
models = {
    "student1": "bylang/student1_distilgpt2",
    "student1_base": "bylang/student1_base_distilgpt2",
    "student2": "bylang/student2_gemma",
    "student2_base": "bylang/student2_base_gemma",
}

pipelines_dict = {}


#modellerin yükleme döngüsü
"""
Her modeli sırayla GPU'lara yükler ve pipeline oluşturur.
GPU indeksini döndürerek modelleri farklı GPU'lara dağıtır.
"""
gpu_idx = 0
for key, model_name in models.items():
    print(f"{key} modeli GPU {gpu_idx}'ye yükleniyor")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token  # Pad token'ı EOS olarak ayarlar

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",   # Accelerate kütüphanesi modelleri otomatik dağıtır
        torch_dtype=torch.float16  # Hafıza tasarrufu için float16 precision kullanır
    )

    # Pipeline oluştururken device belirtmiyoruz, model zaten device_map'a göre ayarlandı
    pipelines_dict[key] = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=80,  # Üretilecek maksimum yeni token sayısı
        do_sample=True,  # Rastgele örnekleme etkin
        temperature=0.7,  # Çıktının rastgelelik seviyesi
        top_p=0.9,  # Top-p (nucleus) örnekleme
        repetition_penalty=1.2,  # Tekrar eden token'lara ceza
        no_repeat_ngram_size=3  # Tekrarlanmayan n-gram boyutu
    )

    # GPU indeksini sıradaki GPU'ya geçir
    gpu_idx = (gpu_idx + 1) % n_gpus



"""
hugginface'ten hazırladığım davranış verilerinin soru kısımlarını ayırır"""
dataset = load_dataset("bylang/behavior_data", split="train")
questions = dataset["question"]



"""
Her model için sorulara cevap üretir.
Pipeline kullanarak text generation yapar ve cevabı çıkarır.
"""
data = {"question": questions}

for key, pipe in pipelines_dict.items():
    answers = []
    for q in questions:
        # Pipeline, modelin device_map'ına göre GPU kullanır
        output = pipe(f"<s>[INST] {q} [/INST] ")[0]["generated_text"]
        # Üretilen metinden [INST] sonrası kısmı alır ve cevabı çıkarır
        ans = output.split("[INST]")[1].split("[/INST]")[-1].strip()
        answers.append(ans)
    data[f"{key}_answer"] = answers



"""
Üretilen verileri DataFrame'e dönüştürür ve CSV dosyasına kaydeder.
"""
df_answers = pd.DataFrame(data)
df_answers.to_csv("student1_answers_clean_gpu_2.csv", index=False)
print(df_answers.head())

In [ ]:
"""
Bu notebook, öğrenci modellerinin performansını değerlendirmek için kullanılır.
Model cevapları ile gerçek referans cevapları arasındaki benzerliği çeşitli metriklerle ölçer.
Kullanılan metrikler:
- BLEU Score: N-gram eşleşmeleri üzerinden çeviri kalitesini ölçer. Daha uzun n-gram'lar için daha yüksek puan verir.
- Jaccard Similarity: Kelime kümelerinin kesişiminin birleşimine oranı. Kelime bazlı benzerliği hesaplar.
- TF-IDF Cosine Similarity: TF-IDF vektörleri arasındaki kosinüs açısı ile benzerliği ölçer. Kelime sıklığı ve önemini dikkate alır.
"""

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


"""
Gerekli kütüphanelerin import edilmesi.
pandas: Veri manipülasyonu için.
numpy: Sayısal işlemler için.
matplotlib.pyplot: Grafikler için.
datasets: Hugging Face dataset'lerini yüklemek için.
sklearn: TF-IDF ve kosinüs benzerliği için.
nltk: BLEU skoru için.
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datasets import load_dataset

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download("punkt")




"""
Konfigürasyon ayarları.
Model cevaplarının bulunduğu CSV dosyasının yolu ve sütun isimleri.
Soru ve referans sütunlarının isimleri.
"""
ANSWERS_CSV = "/kaggle/input/datasetss/all_answers.csv" # Model cevaplarının CSV dosya yolu

MODEL_COLUMNS = {
    "student1": "student1_answer",
    "student1_base": "student1_base_answer",
    "student2": "student2_answer",
    "student2_base": "student2_base_answer",
}

QUESTION_COLUMN = "question"
REFERENCE_COLUMN = "answer"




"""
Model cevaplarının güvenli bir şekilde yüklenmesi.
CSV dosyasından veri okunur, hatalı satırlar atlanır.
"""
df_answers = pd.read_csv(
    ANSWERS_CSV,
    sep=None,
    engine="python",
    encoding="utf-8",
    encoding_errors="replace",
    on_bad_lines="skip"
).fillna("")

print("Model answers loaded:", df_answers.shape)




"""
Gerçek referans verilerinin Hugging Face dataset'inden yüklenmesi.
behavior_data dataset'i kullanılır.
"""
behavior_ds = load_dataset("bylang/behavior_data", split="train")
df_gt = behavior_ds.to_pandas()[[QUESTION_COLUMN, REFERENCE_COLUMN]].fillna("")

print("Ground truth loaded:", df_gt.shape)




"""
Model cevapları ile referans verilerinin soru bazlı birleştirilmesi.
Inner join ile sadece ortak sorular dahil edilir.
"""
df = pd.merge(
    df_answers,
    df_gt,
    on=QUESTION_COLUMN,
    how="inner"
)

print("Merged dataset:", df.shape)
display(df.head())




"""
Metrik fonksiyonları tanımlanır.
Bu fonksiyonlar, model cevaplarının referanslarla karşılaştırılması için kullanılır.
"""

smooth = SmoothingFunction().method1


def bleu_score(ref, hyp):
    """
    BLEU Score metriği.
    N-gram eşleşmeleri üzerinden çeviri kalitesini ölçer.
    1-gram'dan 4-gram'a kadar olan eşleşmeleri değerlendirir.
    Smoothing function ile sıfır bölme önlenir.
    """
    return sentence_bleu(
        [ref.split()],
        hyp.split(),
        smoothing_function=smooth
    )

def jaccard_similarity(ref, hyp):
    """
    Jaccard Similarity metriği.
    Kelime kümelerinin kesişiminin birleşimine oranı.
    Kelime bazlı benzerliği hesaplar.
    Her iki metin boşsa 1.0 döndürür.
    """
    ref_set = set(ref.split())
    hyp_set = set(hyp.split())
    if not ref_set and not hyp_set:
        return 1.0
    return len(ref_set & hyp_set) / len(ref_set | hyp_set)



"""
TF-IDF Cosine Similarity fonksiyonu.
Referans ve hipotez metinlerini TF-IDF vektörlerine dönüştürür.
Aralarındaki kosinüs benzerliğini hesaplar.
"""

def tfidf_cosine_scores(references, hypotheses):
    """
    TF-IDF Cosine Similarity metriği.
    TF-IDF vektörleri arasındaki kosinüs açısı ile benzerliği ölçer.
    Kelime sıklığı ve önemini dikkate alır.
    Diagonal elemanlar alınarak her referans-hipotez çifti için skor döndürür.
    """
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(references + hypotheses)

    ref_vecs = tfidf[:len(references)]
    hyp_vecs = tfidf[len(references):]

    return cosine_similarity(ref_vecs, hyp_vecs).diagonal()



"""
Ana değerlendirme döngüsü.
Her model için metrikleri hesaplar.
BLEU, Jaccard ve TF-IDF Cosine skorlarını ortalar.
"""

results = {}

refs = df[REFERENCE_COLUMN].tolist()

for model_name, col in MODEL_COLUMNS.items():
    print(f"Evaluating {model_name}...")

    hyps = df[col].tolist()

    exact = []
    bleu = []
    jaccard = []

    for r, h in zip(refs, hyps):
        bleu.append(bleu_score(r, h))
        jaccard.append(jaccard_similarity(r, h))

    cosine = tfidf_cosine_scores(refs, hyps)

    results[model_name] = {
        "BLEU": np.mean(bleu),
        "Jaccard": np.mean(jaccard),
        "TFIDF-Cosine": np.mean(cosine),
    }




"""
Sonuçların tablo olarak gösterilmesi ve CSV'ye kaydedilmesi.
"""
results_df = pd.DataFrame(results).T
print("\nFINAL RESULTS")
display(results_df)

results_df.to_csv("final_model_evaluation_results.csv")



"""
Sonuçların görselleştirilmesi.
Bar grafikler, histogramlar ve boxplot ile metrik dağılımları gösterilir.
"""

plt.figure(figsize=(14, 6))
results_df.plot(kind="bar")
plt.title("Student vs Base Model Evaluation (Ground Truth: behavior_data)")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.grid(True)
plt.tight_layout()
plt.show()

# metrik başına histogram
for metric in results_df.columns:
    plt.figure()
    results_df[metric].plot(kind="hist", bins=10)
    plt.title(f"Distribution of {metric}")
    plt.xlabel(metric)
    plt.grid(True)
    plt.show()

# Boxplot
plt.figure(figsize=(12, 6))
results_df.boxplot()
plt.title("Metric Distribution Across Models")
plt.grid(True)
plt.show()

print("\nEvaluation completed successfully.")